# 銷售行為分析

你剛接手超商的店長職位，為了想要了解顧客的銷售行為，打算打造一個系統來搜集並且分析資料。

在這個系統中，你會紀錄每個顧客的性別與年齡層，以及所購買的所有商品，希望能夠透過這些資料來進行銷售行為與顧客的分析。

In [223]:
# 商品結構
# p001 為商品編號
items = {
    "p001": {
        "name": "番茄",
        "categories": ["fruit", "vegetable"],
        "price": 32,
        "calories": 123
    },
    "p002": {
        "name": "西瓜",
        "categories": ["fruit"],
        "price": 123,
        "calories": 22        
    }
}


# 訂單結構
# age: 10(10歲以下), 20(10-20歲), 30, 40, 50, 60, 99(目測超過60歲), -1(無法目測歲數)
# gender: 0(男生), 1(女生)
orders = [ #list + dict
    {
        "time": "2020-03-01 12:00:12",
        "customer": {
            "age": 20, 
            "gender": 0,
        },
        "items": [
            "p001",  # 編號 p001 買 1 個
            "p002",  # 編號 p002 買 3 個
            "p002",
            "p002",
        ]
    },
    {
        "time": "2020-03-01 12:00:12",
        "customer": {
            "age": -1,
            "gender": 0,
        },
        "items": [
            "p001",  # 編號 p001 買 1 個
        ]
    },
]

#order會以相同結構一直增加，適合會大量增生的資料
#dict可以快速對應資料

此為產生假資料的腳本，請務必執行～

In [224]:
import string
from datetime import datetime
from random import randint, choice

NUM_ITEMS = 50
NUM_ORDERS = 10000
GENDERS = [0, 1]
AGES = [-1, 99, 10, 20, 30, 40, 50, 60]
CATEGORIES = ['fruit', 'vegetable', 'drink', 'meat', 'bread']
items = { 
    "p{:03d}".format(idx): {
        "name": ''.join([choice(string.ascii_letters) for _ in range(randint(5, 10))]),
        "categories": [choice(CATEGORIES) for _ in range(randint(1, 3))],
        "price": randint(10, 500),
        "calories": randint(50, 1000)
    } 
    for idx in range(NUM_ITEMS)
}

now = datetime.now().timestamp()
orders = [
    {
        "time": datetime.fromtimestamp(now - randint(0, 86400 * 30)).strftime('%Y-%m-%d %H:%M:%S'),
        "customer": {
            "age": choice(AGES),
            "gender": choice(GENDERS),
        },
        "items": [choice(list(items.keys())) for _ in range(randint(1, 10))]
    }
    for idx in range(NUM_ORDERS)
]

## Q1 所有訂單的銷售總額？

In [225]:
# imperative
total_price = 0
for order in orders:
    order_items = order['items']
    for item_id in order_items:
        total_price = total_price + items[item_id]['price']
print(total_price)

13527209


In [226]:
#decalarative
total_price_d = sum([
    items[item_id]['price']
    for order in orders
    for item_id in order['items']
])
print(total_price_d)

13527209


## Q2 最熱賣的十項商品為何？

In [227]:
# imperative
item_id_list = []
for order in orders:
    item_id_list = item_id_list + order["items"]

count_item_id_list = []
for item_id in set(item_id_list):
    count_item_id = (item_id_list.count(item_id), item_id)
    count_item_id_list.append(count_item_id)

count_item_id_list.sort(reverse=True)
for count, item_id in count_item_id_list[:10]:
    print(items[item_id]["name"], count)
print(count_item_id)



OdockpnMg 1159
wvznUqol 1153
JXMCrj 1149
MTXazI 1146
ZojxKbQAW 1139
TNjVz 1136
KwHvGsGJY 1136
xUbemZo 1135
RDXyNdEJMA 1135
jRZHa 1120
(1062, 'p035')


In [228]:
# declarative
item_id_list = [
    item_id
    for order in orders
    for item_id in order["items"]
]
count_item_id_list = [
    (item_id_list.count(item_id), item_id)
    for item_id in set(item_id_list)
]
# immutable
sorted_count_item_id_list = sorted(count_item_id_list, reverse=True)
result = [
    (items[item_id]["name"], count)
    for count, item_id in sorted_count_item_id_list[:10]
]
print(result)

[('OdockpnMg', 1159), ('wvznUqol', 1153), ('JXMCrj', 1149), ('MTXazI', 1146), ('ZojxKbQAW', 1139), ('TNjVz', 1136), ('KwHvGsGJY', 1136), ('xUbemZo', 1135), ('RDXyNdEJMA', 1135), ('jRZHa', 1120)]


In [229]:
# collections: counter & most.common

from collections import Counter

item_id_list = []
for order in orders:
    item_id_list = item_id_list + order['items']

acc = Counter(item_id_list)
result = acc.most_common()
for x in result[:10]:
    print(items[x[0]]['name'],x[1])

OdockpnMg 1159
wvznUqol 1153
JXMCrj 1149
MTXazI 1146
ZojxKbQAW 1139
TNjVz 1136
KwHvGsGJY 1136
xUbemZo 1135
RDXyNdEJMA 1135
jRZHa 1120


## Q3 總熱量最低的訂單總熱量為何？

In [230]:
#imperative
order_customer_calories_list = []
for order in orders:
    total_calories = 0
    for item_id in order['items']:
        total_calories = total_calories + items[item_id]['calories']

    order_customer_calories = (total_calories, order['customer']['age'],order['customer']['gender'])
    order_customer_calories_list.append(order_customer_calories)

print(min(order_customer_calories_list))

(58, -1, 0)


In [231]:
# declarative 把imperative從由內開始改寫到declarative外

total_calories = sum([
    items[item_id]['calories']
    for item_id in order['items']
])

order_customer_calories_list = [
    (
        sum([
            items[item_id]['calories']
            for item_id in order['items']
        ]),
        order['customer']["age"],
        order['customer']["gender"]
    )
    for order in orders
]
print(min(order_customer_calories_list))

(58, -1, 0)


## Q4 青少年(10-30歲) 最愛的商品為何？

In [232]:
# imperative
# tuple 比較會從第一格開始比 要比較的對象要放最前面

item_id_list = []
for order in orders:
    if order['customer']['age'] == 20 or order['customer']['age'] == 30:
        for item_id in order['items']:
            item_id_list.append(item_id)

item_id_count_list = []
for item_id in set(item_id_list):
    item_id_count = (item_id_list.count(item_id), item_id)
    item_id_count_list.append(item_id_count)

count, item_id = max(item_id_count_list)
print(items[item_id]['name'])

cGwcA


In [233]:
# declarative
item_id_list = [
    item_id
    for order in orders
    for item_id in order['items']
    if order['customer']['age'] == 20 or order['customer']['age'] == 30
]

item_id_count_list = [
    (item_id_list.count(item_id),item_id)
    for item_id in set(item_id_list)
]

count, item_id = max(item_id_count_list)
print(items[item_id]['name'])

cGwcA


## Q5 男性最愛的類別為何？


In [234]:
# imperative
category_list = []
for order in orders:
    if order['customer']['gender'] == 0:
        for item_id in order['items']:
            for category in items[item_id]['categories']:
                category_list.append(category)
category_count_list = []
for category in set(category_list):
    category_count = (category_list.count(category),category)
    category_count_list.append(category_count)

print(max(category_count_list))

(15473, 'meat')


In [235]:
# declarative
category_list = [
    category

    for order in orders
    for item_id in order['items']
    for category in items[item_id]['categories']

    if order['customer']['gender'] == 0
]

category_count_list = [
    (category_list.count(category),category)
    for category in set(category_list)
]

print(max(category_count_list))

(15473, 'meat')


In [236]:
# counter

category_list = [
    category

    for order in orders
    for item_id in order['items']
    for category in items[item_id]['categories']

    if order['customer']['gender'] == 0
]
category_count_list = Counter(category_list)
print(category_count_list.most_common()[0])

('meat', 15473)


## Q6 女性訂單中，總金額最大值為何？

In [237]:
female_orders = []
for order in orders:
    if order['customer']['gender'] == 1:
        female_orders.append(order)

order_price_items_list = []
for order in female_orders:
    total_price = 0
    for item_id in order['items']:
        total_price = total_price + items[item_id]['price']
    
    order_price_items = (total_price, order['items'])
    order_price_items_list.append(order_price_items)

total_price, order_items = max(order_price_items_list)
print(total_price)
for item_id in order_items:
    print(items[item_id]['name'], items[item_id]['price'])
    

3734
ZojxKbQAW 488
BTgBTeS 357
GWoKO 497
KwHvGsGJY 410
cGwcA 196
OdockpnMg 434
JLDgzLABrF 416
JLDgzLABrF 416
MahLSMRVO 283
zIwbq 237


list comprehension 變數不會被儲存 

In [238]:
female_orders = [
    order

    for order in orders
    if order["customer"]["gender"] == 1
]

order_price_items_list = [
    (
        sum ([
             items[item_id]['price']
             for item_id in order["items"]
        ]),
        order['items']
    )
    for order in female_orders
]

total_price, order_items = max(order_price_items_list)
print(total_price)
for item_id in order_items:
    print(items[item_id]["name"], items[item_id]["price"])

3734
ZojxKbQAW 488
BTgBTeS 357
GWoKO 497
KwHvGsGJY 410
cGwcA 196
OdockpnMg 434
JLDgzLABrF 416
JLDgzLABrF 416
MahLSMRVO 283
zIwbq 237


## Q7 飲料類別中，各年齡層的消費力為何？
消費力 = 消費金額總數，假設現在有三筆訂單：

1. age(20), 番茄、牛奶(100)
2. age(10), 奶茶(30)
3. age(10), 果汁(15)、紅茶(20)

因為番茄沒有飲料的類別，所以濾掉不計

* age: 20 的消費力 = 100
* age: 10 的消費力 = 30 + 15 + 20 = 65

In [239]:
# item -> categroy, customer -> age, item -> price

for age_level in AGES:
#for age_level in [10, 20, 30, 40, 50, 60, 99, -1]:
    total_price = 0    
    for order in orders:
        if order['customer']['age'] == age_level:
            for item_id in order['items']:
                if 'drink' in items[item_id]['categories']:
                    total_price = total_price + items[item_id]['price']

    print(age_level, total_price)

-1 438368
99 424855
10 431473
20 420035
30 449079
40 437402
50 441072
60 402696


In [240]:

result= [
    (
        age_level,
        sum([
            sum([
            items[item_id]['price']
            for item_id in order['items']
            if 'drink' in items[item_id]['categories']
            ])
            for order in orders
            if order['customer']['age'] == age_level
        ])
    )
    for age_level in AGES
        
]
print(result)

[(-1, 438368), (99, 424855), (10, 431473), (20, 420035), (30, 449079), (40, 437402), (50, 441072), (60, 402696)]


## Q8 商品 p000 最常跟哪個商品一起購買？

In [241]:
item_id_list =[]
for order in orders:
    if 'p000' in order['items']:
        for item_id in set(order['items']):
            if item_id != 'p000':
                item_id_list.append(item_id)
item_id_count_list = []                
for item_id in set(item_id_list):
    item_id_count = (item_id_list.count(item_id),item_id)
    item_id_count_list.append(item_id_count)

count, item_id = max(item_id_count_list)
print(items[item_id]['name'], count)

IlloM 135


In [242]:
item_id_list =[
    item_id

    for order in orders
    if 'p000' in order['items']

    for item_id in set(order['items'])
    if item_id != 'p000'
]

item_id_count_list = [
    (item_id_list.count(item_id),item_id)
    for item_id in set(item_id_list)
] 

count, item_id = max(item_id_count_list)
print(items[item_id]["name"], count)

IlloM 135


In [243]:
# collections: counter & most.common

item_id_list =[
    item_id

    for order in orders
    if 'p000' in order['items']

    for item_id in set(order['items'])
    if item_id != 'p000'
]

item_id_count_list = Counter(item_id_list)
result = item_id_count_list.most_common()
for x in result[:1]:
    print(items[x[0]]["name"], x[1])



IlloM 135


## Q9 每個商品的男性購買力與女性購買力各自為何？

In [244]:
# order -> item_id
# order -> customer -> gender
# item -> price(opational)
item_id_gender_list = []
for order in orders:
    for item_id in order['items']:
        item_id_gender = (order['customer']['gender'],item_id)
        item_id_gender_list.append(item_id_gender)

# item_id = 'p001'
# gender = 0
# print(item_id_gender_list.count((gender, item_id)) * items[item_id]["price"])

def calculate_power(item_id_gender_list, gender, item_id):
    return item_id_gender_list.count((gender, item_id))*  items[item_id]['price']
    

In [245]:
calculate_power(item_id_gender_list, 1, 'p002')

49000

In [246]:
calculate_power(item_id_gender_list, 0, 'p003')

153045

In [247]:
item_id_gender_list = [
    (order["customer"]["gender"], item_id)
    for order in orders
    for item_id in order["items"]
]
        
def calculate_power(item_id_gender_list, gender, item_id):
    return item_id_gender_list.count((gender, item_id)) * items[item_id]["price"]

In [248]:
calculate_power(item_id_gender_list, 1, 'p002')

49000

In [249]:
calculate_power(item_id_gender_list, 0, 'p003')

153045